___
# Atividade: Regressão Linear Simples
___

## Aula 25

**Preparo Prévio:**
1. Montogmery. Seção 6-2 - Simple Linear Regression
1. Magalhães e Lima, seção 9.5. Regressão Linear Simples


Referência: http://www.statsmodels.org/dev/diagnostic.html

Referência: [http://connor-johnson.com/2014/02/18/linear-regression-with-python/](http://connor-johnson.com/2014/02/18/linear-regression-with-python/)

**Hoje:**
1. Entender o conceito de regressão linear

___

## Renda vs CO2

Foram coletados dois dados do site https://www.gapminder.org/:
1. Emissão de CO2 per capita
1. Renda per capita

As linhas representam os países e as colunas representam o ano. 

Como os arquivos estão separados, será preciso juntá-los cruzando o país.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
#Leitura dos arquivos em Excel
co2 = pd.read_excel('indicator CDIAC carbon_dioxide_emissions_per_capita.xlsx')
income = pd.read_excel('indicator gapminder gdp_per_capita_ppp.xlsx')

In [ ]:
#É possível verificar que cada linha representa um país e as colunas representam o ano
co2.head(2)

In [ ]:
#O mesmo se aplica a renda.
income.head(2)

___
### Inner Join

Vamos agora juntar as duas tabelas via país, selecionar apenas o ano de 2010 e remover os NaNs.

Ao final vamos fazer o gráfico de dispersão das duas variáveis.

**Sugestão**: pesquise sobre a função DataFrame.join(), pode ser muito útil no futuro.

In [ ]:
#Cruza as duas tabelas via país
co2.set_index('CO2 per capita', inplace=True)
income.set_index('GDP per capita', inplace=True)

df = co2.join(income, how='inner', lsuffix='_co2', rsuffix='_income')

#Seleciona o ano de 2010 e remove os NaNs
df = df[['2010_co2','2010_income']].dropna()

#Transforma a renda na escala de milhares de dólares
df['2010_income'] /= 1000


___
### Problema

Considere que o objetivo aqui seja explicar/prever a emissão de gás carbono (CO2) per capita de um país em função da renda (PIB) per capita.

Por conta disso, vamos considerar CO2 como variável dependente (eixo y) e renda como explicativa (eixo x).

In [ ]:
x = df['2010_income']
y = df['2010_co2']
plt.scatter(x,y);
plt.xlabel("x: 2010 income");
plt.ylabel("y: 2010 co2 per capita");

___
### Regressão

Vamos tentar agora ajustar um modelo sobre os dados. A primeira tentativa será ajustar um reta:

$$y_i=\beta_0+\beta_1x_i+\epsilon_i$$


Esse modelo é muito parecido com o visto na Aula 05, com algumas mudanças:
1. Agora utilizaremos $\beta_i$ para se referir aos coeficientes.
1. Existe um termo $\epsilon_i$ para representar os resíduos.

O primeiro passo agora é calcular os valores da regressão. Existem diversas formas de estimar os $\beta$s, vamos utilizar o método de **Mínimos Quadrados Ordinários (MQO ou OLS - Ordinary Least Squares em inglês)**. 

$$\hat{\beta}_0=\bar{y}-\hat{\beta}_1\bar{x}$$

$$\hat{\beta}_1=\frac{S_{XY}}{S_{XX}}=\frac{\sum_{i=1}^n(x_i-\bar{x})(y_i-\bar{y})}{\sum_{i=1}^n(x_i-\bar{x})^2}$$

**Obs: Acompanhar e anotar o desenvolvimento no quadro.**
___

### Tarefa 1 - $\hat{\beta}_1$ e  $\hat{\beta}_0$

Calcule os valores de $\bar{x}$ e $\bar{y}$. Calcule também $S_{XX}$ e $S_{XY}$, em seguida calcule o $\hat{\beta}_1$ e o $\hat{\beta}_0$

In [ ]:
#B0 e B1

yb = y.mean()
xb = x.mean()

Sxy = ((x -xb)*(y -yb)).sum()
Sxx = ((x -xb)**2).sum()

b1 = Sxy/Sxx
b0 = yb - b1*xb

print("beta0: {0}".format(b0))
print("beta1: {0}".format(b1))


### Verificando os resultados da regressão



Vamos contrastar os resultados previstos pela regressão com os dados.

In [ ]:
x_v = np.linspace(x.min(), x.max(), 500)
y_v = b0 + b1*x_v

In [ ]:
plt.plot(x_v,y_v, color="r") # resultados da regressão
plt.scatter(x, y) # dados

### Tarefa 2 - Resíduos 

Agora calcular o vetor de resíduos, dado por:
$$\epsilon_i=y_i-(\hat{\beta}_0+\hat{\beta}_1x_i)$$

**Verifique a normalidade dos resíduos. Parece uma Normal?**

In [ ]:
 # Escreva sua fórmula dos resíduos aqui

 # Verifique graficamente  se é uma normal - como fazer isso? Lembre-se do probplot

### Teste de hipóteses para os coeficientes 

Agora que já aprendemos Teste de Hipóteses, podemos verificar se os Betas são realmente relevantes. para tal vamos testar:

$$H_0: \beta_i=0$$
$$H_1: \beta_i\neq0$$



 Verifique o **valor-p** para os betas e verifique se rejeitamos ou não $H_0$. Utilize $\alpha=5\%$. Verifique os resultados da regressão
 
 Responda depois da seção abaixo

___
### Usando statsmodels.OLS

Agora que estão familiarizados com algumas medidas, Rode a função OLS() da biblioteca statsmodel e interprete:

In [ ]:
import numpy as np
import statsmodels.api as sm


xc = sm.add_constant(x)
#xc = x
model = sm.OLS(y,xc)
results = model.fit()
results.summary()

### Plot da reta ajustada

**Tarefa 3**

Usando os resultados da equação obtida via `statsmodels`, plote novamente a reta ajustada sobre os pontos

In [ ]:
x_vc = sm.add_constant(x_v)
y_vc = results.predict(x_vc)
plt.plot(x_v, y_vc, color="r")
plt.scatter(x,y);

### Resíduos

Plot da normalidade dos resíduos. O mesmo plot dos resíduos que foi feito acima pode ser facilitado se usarmos o atributo `resid` dos resultados da regressão

___



**Tarefa 4**: Você consegue interpretar alguns dos resultados acima? Você acha que os resíduos formam uma normal? Fale um pouco sobre a homocedasticidade do modelo.

In [ ]:
# Descomente para funcionar
#stats.probplot(results.resid, dist="norm", plot=plt);

## Resumo dos testes da regressão


Os resumo dos resultados da regressão traz informações que nos permitem avaliar a qualidade do ajuste e a validade de:

* Normalidade dos resíduos
* Testes de hipóteses dos resíduos

### Coeficiente de determinação $R^2$

É uma medida de quão bem uma regressão capta a variação presente nos dados. 


$$ R^2 = 1 - \frac{SS_E}{SS_T}$$


Para calcular esta fórmula precisamos das relações:

$$SQRes=SS_{E}=\sum\limits^{n}_{i=1}(y_i-\hat{y}_i)^2=\sum\limits_{i=1}^{n}\epsilon^2_{i}$$


$$SQT=SS_{T}=\sum\limits^{n}_{i=1}(y_i-\bar{y})^2$$





### p-values da estatística t

Existem para cada coeficiente $\beta_i$

Testam a seguinte hipótese:

$H_0: \beta_i = 0$

$H_1: \beta_i \neq 0$

Um *p-value* baixo permite **rejeitar** a hipótese $H_0$ que $\beta_i = 0$




### $R^2$ ajustado

$N$ é o número de dados
$P$ é o número de preditores

$Rajustado^2 = R^2 - (1-R^2)\frac{P}{N-P-1}$

Penaliza preditores que não acrescentam poder preditivo significativo

### Estatística F

Testa as seguintes hipóteses:

$H_0: \beta_i = 0$, para todo $i > 0$

$H_1: \beta_i \neq 0$, para algum $i > 0$


Ou seja: compara o modelo obtido versus um modelo só com intercepto

### Teste Omnibus

Testa a normalidade dos resíduos

Se $Pr(Omnibus)$ for muito baixo, existe evidência de que os resíduos **não são** distribuídos normalmente

### Teste Jarque-Bera

Outro teste de normalidade.

$H_0:$ a distribuição dos resíduos é normal

$H_1:$ a distribuição dos resíduos não é normal


### Teste Durbin-Watson

Testa autocorrelação dos resíduos.

Varia na faixa $[0, 4]$

* Um valor próximo de $2$ sugere que não há autocorrelação dos resíduos
* Um valor menor que $2$ sugere **correlação positiva** dois resíduos
* Um valor maior que $2$ sugere **correlação negativa** dois resíduos





### Tarefa 5 - Análise dos valores p

O que os valores *p* da regressão dizem a respeito dos *betas?*

**R.:**

### Tarefa 6 - Análise da estatística F

O que o valor da estatística F diz sobre a qualidade da regressão? 

Diga qual hipótese nula e alternativa $Prob(F)$ avalia

**R.:**

### Tarefa 7 - Análise do $R^2$

O que o valor de $R^2$ obtido diz sobre o poder explicativo da regressão obtida via OLS?

**R.:**

### Tarefa 8 - Análise de normalidade do resíduo

O que os testes *Omnibus* e *Jarque-Bera* dizem a respeito da normalidade do resíduo? 

**R.:**

### Tarefa 9 - Análise de homocedasticidade

Verifique visualmente se a hipótese de homocedasticidade é válida.

**R.:**